<font size=50 color=darkblue>Map Coloring using Big-M Transformation</font>

# Problem modelling in LP format

## Import necessary modules
- Import function `read_model` from DOcplex to read the Map-Coloring MILP from a temporary file
- Module `tempfile` is imported to create the temporary file
- Import function `remove` to delete the temporary file after reading it (required for python version < 3.12)

In [ ]:
from docplex.mp.model_reader import read_model
import tempfile
from os import remove

## Map-Coloring MILP using Big-M Transformation <font size=3>(variables are colored blue)</font>
**Minimize**
### $$\color{blue}x_\textbf{MAX}$$
**Subject to**
### \begin{align*}
{\color{blue}x_\textbf{MAX}} - {\color{blue}x_{(v)}}&\ge 0,\qquad&\forall v\in \mathcal{V}\\
({\color{blue}x_{(u)}} - {\color{blue}x_{(v)}} + M {\color{blue}t_{(u,v)}}&\ge 1) \wedge ({\color{blue}x_{(v)}} - {\color{blue}x_{(u)}} - M {\color{blue}t_{(u,v)}}\ge 1 - M),\qquad&\forall (u,v)\in \mathcal{E}\\
{\color{blue}x_{(v)}}&\in \{0,...,3\},\qquad&\forall v\in \mathcal{V}\\
{\color{blue}t_{(u,v)}}&\in \{0,1\},\qquad&\forall (u,v)\in \mathcal{E}\\
{\color{blue}x_\textbf{MAX}}&\in\mathbb{R}
\end{align*}

## Write the Map-Coloring MILP model (in LP format), which is assigned to the variable `colr_str`
### Notes
- The big-M parameter $M$ here is set to 1000.
- Integer variables are declared in section `Generals` in the LP format

In [ ]:
colr_str = '''
Minimize
 obj: xMAX
 
Subject To
 xMAX(Belgium)     : xMAX - x(Belgium)     >= 0
 xMAX(Denmark)     : xMAX - x(Denmark)     >= 0
 xMAX(France)      : xMAX - x(France)      >= 0
 xMAX(Germany)     : xMAX - x(Germany)     >= 0
 xMAX(Luxembourg)  : xMAX - x(Luxembourg)  >= 0
 xMAX(Netherlands) : xMAX - x(Netherlands) >= 0
 dis_colr_a(Germany,Belgium)     : x(Germany)     - x(Belgium)     + 1000 t(Germany,Belgium)     >= 1
 dis_colr_b(Germany,Belgium)     : x(Belgium)     - x(Germany)     - 1000 t(Germany,Belgium)     >= -999
 dis_colr_a(Germany,Denmark)     : x(Germany)     - x(Denmark)     + 1000 t(Germany,Denmark)     >= 1
 dis_colr_b(Germany,Denmark)     : x(Denmark)     - x(Germany)     - 1000 t(Germany,Denmark)     >= -999
 dis_colr_a(Germany,France)      : x(Germany)     - x(France)      + 1000 t(Germany,France)      >= 1
 dis_colr_b(Germany,France)      : x(France)      - x(Germany)     - 1000 t(Germany,France)      >= -999
 dis_colr_a(Germany,Netherlands) : x(Germany)     - x(Netherlands) + 1000 t(Germany,Netherlands) >= 1
 dis_colr_b(Germany,Netherlands) : x(Netherlands) - x(Germany)     - 1000 t(Germany,Netherlands) >= -999
 dis_colr_a(Germany,Luxembourg)  : x(Germany)     - x(Luxembourg)  + 1000 t(Germany,Luxembourg)  >= 1
 dis_colr_b(Germany,Luxembourg)  : x(Luxembourg)  - x(Germany)     - 1000 t(Germany,Luxembourg)  >= -999
 dis_colr_a(Belgium,France)      : x(Belgium)     - x(France)      + 1000 t(Belgium,France)      >= 1
 dis_colr_b(Belgium,France)      : x(France)      - x(Belgium)     - 1000 t(Belgium,France)      >= -999
 dis_colr_a(Belgium,Netherlands) : x(Belgium)     - x(Netherlands) + 1000 t(Belgium,Netherlands) >= 1
 dis_colr_b(Belgium,Netherlands) : x(Netherlands) - x(Belgium)     - 1000 t(Belgium,Netherlands) >= -999
 dis_colr_a(Belgium,Luxembourg)  : x(Belgium)     - x(Luxembourg)  + 1000 t(Belgium,Luxembourg)  >= 1
 dis_colr_b(Belgium,Luxembourg)  : x(Luxembourg)  - x(Belgium)     - 1000 t(Belgium,Luxembourg)  >= -999
 dis_colr_a(France,Luxembourg)   : x(France)      - x(Luxembourg)  + 1000 t(France,Luxembourg)   >= 1
 dis_colr_b(France,Luxembourg)   : x(Luxembourg)  - x(France)      - 1000 t(France,Luxembourg)   >= -999

Bounds
 x(Belgium) <= 3
 x(Denmark) <= 3
 x(France) <= 3
 x(Germany) <= 3
 x(Luxembourg) <= 3
 x(Netherlands) <= 3

Binaries
 t(Germany,Belgium) t(Germany,Denmark) t(Germany,France) t(Germany,Netherlands)
 t(Germany,Luxembourg) t(Belgium,France) t(Belgium,Netherlands)
 t(Belgium,Luxembourg) t(France,Luxembourg)

Generals
 x(Belgium) x(Denmark) x(France) x(Germany) x(Luxembourg) x(Netherlands)
 
End
'''

## To import the MILP model to DOcplex
- Store the model to a temporary file
- Have DOcplex read the model from the temporary file

In [ ]:
# Store string to tmp file
with tempfile.TemporaryFile(delete=False) as tmp:
    tmp.write(colr_str.encode('utf-8'))
    tmp.close()
    # Have DOcplex read the string
    colr_MILP = read_model(filename=tmp.name, model_name='Map Coloring')
    # Delete the temporary file
    remove(tmp.name)

## Summarize the model

In [ ]:
colr_MILP.print_information()

## Solve the MILP and display the result

In [ ]:
colr_sol = colr_MILP.solve()
if colr_sol:
    colr_sol.display()

# Result Visualization

## Import visualization modules
- `igraph`, `matplotlib`, `re`

In [ ]:
import igraph as ig
import matplotlib.pyplot as plt
import re

## Extract country list `V` and neighborship `E` from the MILP model

In [ ]:
V, E = [], []
for var in colr_MILP.iter_variables():
    if var.name.startswith('x('):
        V.append(var.name[2:-1])
    elif var.name.startswith('t('):
        v1, v2 = re.findall('t\((.+),(.+)\)', var.name)[0]
        E.append((v1, v2))

## For convenience, extract the solution to a dictionary named `sol_x`

In [ ]:
sol_x = {v: int(colr_sol.get_value(f'x({v})')) for v in V}
sol_x

## Enumerate some available colors

In [ ]:
colors = ['black', 'white', 'red', 'blue']

## Instantiate a `Graph` object with module `igraph`
### Notes
- __*Node(s)*__ is/are called __*vertex/vertices*__ in `igraph`
- __*Link(s)*__ is/are called __*edge/edges*__ in `igraph`
- <font color=red>The edge list only contains country names (instead of country indices), so vertices and edges have to be added to the <code><font color=red>Graph</font></code> object separately.</font>
- <font color='red'>In Map-Coloring problems, the links are undirected.</font>

In [ ]:
g = ig.Graph(directed=False)
g.add_vertices(V); g.add_edges(E)

## Visualize the graph

In [ ]:
g.vs['label'] = g.vs['name']
g.vs['label_dist'] = 1
g.vs['size'] = 50
g.vs['color'] = [colors[sol_x[v]] for v in V]

fig, ax = plt.subplots()

p = ig.plot(g, layout=g.layout('fr'), target=ax)
plt.show()